In [1]:
import pandas as pd
import datetime

from plot import remove_begin_end, read_file, get_files

In [12]:
list_of_files = get_files()
data = [remove_begin_end(read_file(filename)) for filename in list_of_files]
data = pd.concat(data).sort_index()
data.index.min(), data.index.max()

(Timestamp('2023-01-12 17:19:39+0100', tz='CET'),
 Timestamp('2023-01-19 10:00:35+0100', tz='CET'))

In [10]:
data_raw = [read_file(filename) for filename in list_of_files]
data_raw = pd.concat(data_raw).sort_index()

In [4]:
len(data), len(data_raw)

(9589, 9658)

In [5]:
#notes = pd.read_csv('data/notes.txt', header=None, sep=' ', parse_dates=[[0, 1]])
#notes = notes.set_index("0_1")
#notes

In [6]:
from scipy.optimize import curve_fit
import numpy as np

def myexp(t, a, b, c):
    return a * np.exp(-t / b) + c

def mylin(t, a, b):
    return a + b * t


def fit_span(series, function, p0):
    start = series.index.min()
    x = series.index - start
    x = x.seconds
    y = series.values
    yinf = series.iloc[-1]
    
    popt, pcov = curve_fit(function, x, y, p0=p0)
    
    return popt, pcov

In [13]:
# riscaldamento 0.25 K/h - 0.44 K/h
# exp b = 1.44625327e+04, 1.04721363e+04

In [15]:
fig.update_yaxes?

In [57]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(template='plotly_dark')
#fig.add_trace(
#    go.Scatter(x=data.index, y=data.temperature, name="temperature"),
#    secondary_y=False,
#)

fig.add_trace(
    go.Scatter(x=data.index, y=data.humidity, name="humidity", line_color='#202AB6'),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=data_raw.index, y=data_raw.temperature, name="temperature r", line_color='#EF553B'),
    secondary_y=False,
)



# Set y-axes titles
fig.update_yaxes(title_text="temperature", secondary_y=False, minor=dict(ticklen=6, tickcolor="white", showgrid=True))
fig.update_yaxes(title_text="humidity", secondary_y=True, showgrid=False, minor=dict(ticklen=6, tickcolor="white", showgrid=False))



fit_timeranges = [
#    (datetime.datetime(2022, 12, 6, 15, 4), datetime.datetime(2022, 12, 6, 22, 0), myexp),
#    (datetime.datetime(2022, 12, 7, 8, 17), datetime.datetime(2022, 12, 7, 16, 57), myexp), 
#    (datetime.datetime(2022, 12, 11, 9, 43), datetime.datetime(2022, 12, 11, 11, 47), mylin),
#    (datetime.datetime(2022, 12, 12, 9, 30), datetime.datetime(2022, 12, 12, 12, 40), mylin)
]
fit_results = []

for start, stop, function in fit_timeranges:
    start = pd.Timestamp(start).tz_localize(LOCAL_TIMEZONE)
    stop = pd.Timestamp(stop).tz_localize(LOCAL_TIMEZONE)
    data_to_fit_mask = (data.index > start) & (data.index < stop)
    data_to_fit = data[data_to_fit_mask]['temperature']

    if function == myexp:
        p0 = (data_to_fit.iloc[0] - data_to_fit.iloc[-1], 15000, data_to_fit.iloc[-1])
    else:
        p0 = (20, 1E-5)
    popt, pcov = fit_span(data_to_fit, function, p0=p0)
    print(popt)

    xspan_seconds = np.linspace(0, (data_to_fit.index.max() - data_to_fit.index.min()).seconds, 100)
    yfit = function(xspan_seconds, *popt)
    xspan_time = data_to_fit.index.min() + pd.TimedeltaIndex(xspan_seconds, 's')
    fig.add_trace(
        go.Scatter(x=xspan_time, y=yfit, showlegend=False, hoverinfo='none'),
    secondary_y=False
)
    
    fit_results.append(popt)

for v in pd.date_range(data.index.min(), data.index.max(), freq='D', normalize=True)[1:]:
    fig.add_vline(v, line_dash='dot', line_color='#777')
    
fig.show()

In [ ]:
import chart_studio.plotly as py
import chart_studio
py.plot(fig)